# Example Data Science Notebook

This notebook demonstrates basic data analysis workflows.

In [ ]:
import sys
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path().resolve().parent / 'src'))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from ds.analyzer import DataAnalyzer
from de.load.loader import DataLoader

print("Imports successful!")

## Load Sample Data

In [ ]:
# Create sample data
np.random.seed(42)
data = {
    'feature1': np.random.randn(100),
    'feature2': np.random.randn(100),
    'feature3': np.random.randn(100),
    'target': np.random.randint(0, 2, 100)
}
df = pd.DataFrame(data)
df.head()

## Exploratory Data Analysis

In [ ]:
analyzer = DataAnalyzer(df)
analyzer.summary_stats()

In [ ]:
# Check for missing values
analyzer.check_missing()

## Visualizations

In [ ]:
# Distribution plot
plt.figure(figsize=(12, 4))
for i, col in enumerate(['feature1', 'feature2', 'feature3'], 1):
    plt.subplot(1, 3, i)
    plt.hist(df[col], bins=20, edgecolor='black')
    plt.title(f'Distribution of {col}')
    plt.xlabel('Value')
    plt.ylabel('Frequency')
plt.tight_layout()
plt.show()

In [ ]:
# Correlation matrix
plt.figure(figsize=(8, 6))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Matrix')
plt.show()

## Prepare Data for ML

In [ ]:
X_train, X_test, y_train, y_test = analyzer.prepare_features('target')
print(f"Training set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")